In [447]:
import regex as re
from pathlib import Path
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import coreferee
from sentence_transformers import SentenceTransformer, util
import getopt
import sys
import pandas as pd
from datasets import load_dataset
import json 
import regex as re
from tqdm import tqdm
import nltk




This is the universal dependency for English: https://universaldependencies.org/en/dep/index.html

### Loading Model

In [448]:
nlp = spacy.load('en_core_web_md')
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe('coreferee')
model = SentenceTransformer("all-MiniLM-L6-v2")

# doc = """
#     Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the golden anniversary with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as Super Bowl L), so that the logo could prominently feature the Arabic numerals 50.
# """
doc = "Politics: U.N. Secretary General Ban Ki-moon; American political leaders John Hancock, John Adams, John Quincy Adams, Rutherford B. Hayes, Theodore Roosevelt, Franklin D. Roosevelt, John F. Kennedy, Al Gore, George W. Bush and Barack Obama; Chilean President Sebastián Piñera; Colombian President Juan Manuel Santos; Costa Rican President José María Figueres; Mexican Presidents Felipe Calderón, Carlos Salinas de Gortari and Miguel de la Madrid; Mongolian President Tsakhiagiin Elbegdorj; Peruvian President Alejandro Toledo; Taiwanese President Ma Ying-jeou; Canadian Governor General David Lloyd Johnston; Indian Member of Parliament Jayant Sinha; Albanian Prime Minister Fan S. Noli; Canadian Prime Ministers Mackenzie King and Pierre Trudeau; Greek Prime Minister Antonis Samaras; Israeli Prime Minister Benjamin Netanyahu; former Pakistani Prime Minister Benazir Bhutto; U. S. Secretary of Housing and Urban Development Shaun Donovan; Canadian political leader Michael Ignatieff; Pakistani Members of Provincial Assembly Murtaza Bhutto and Sanam Bhutto; Bangladesh Minister of Finance Abul Maal Abdul Muhith; President of Puntland Abdiweli Mohamed Ali; U.S. Ambassador to the European Union Anthony Luzzatto Gardner."

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### resolve coreferences

In [449]:

def resolve_coreference(text):
    doc = nlp(text)
    doc_list = list(doc)
    doc._.coref_chains.print()
    resolving_indecies = []
    for _,item in enumerate(doc._.coref_chains):
        resolving_indecies.extend(item)
        
    for word in resolving_indecies:
        new_word = ""
        for index in word:
            if doc[index]._.coref_chains.resolve(doc[index]) is not None:
                temp = []
                for item in doc._.coref_chains.resolve(doc[index]):
                    temp.append(str(item))
                new_word = ", ".join(temp)
            
                doc_list[index] = new_word

    final_doc = []
    for item in doc_list:
        final_doc.append(str(item))
    return " ".join(final_doc)
        

### Preprocess the document

In [450]:
def preprocess_context(doc):
    text = doc.strip()
    text.replace(".", ",")
    resolved_text = resolve_coreference(text)
    resolved_text = resolved_text.strip()
    resolved_text = resolved_text.replace("  ", " ").replace(" ,", ",").replace(";","").replace(" .", ".").replace("\n", "")
    return resolved_text

In [451]:
resolved_doc = preprocess_context(doc)
cleaned_doc = nlp(resolved_doc)
sentences = [one_sentence.text.strip() for one_sentence in cleaned_doc.sents]

In [452]:
sentences

['Politics : U.N. Secretary General Ban Ki-moon  American political leaders John Hancock, John Adams, John Quincy Adams, Rutherford B. Hayes, Theodore Roosevelt, Franklin D. Roosevelt, John F. Kennedy, Al Gore, George W. Bush and Barack Obama  Chilean President Sebastián Piñera  Colombian President Juan Manuel Santos  Costa Rican President José María Figueres  Mexican Presidents Felipe Calderón, Carlos Salinas de Gortari and Miguel de la Madrid  Mongolian President Tsakhiagiin Elbegdorj  Peruvian President Alejandro Toledo  Taiwanese President Ma Ying-jeou  Canadian Governor General David Lloyd Johnston  Indian Member of Parliament Jayant Sinha  Albanian Prime Minister Fan S. Noli  Canadian Prime Ministers Mackenzie King and Pierre Trudeau  Greek Prime Minister Antonis Samaras  Israeli Prime Minister Benjamin Netanyahu  former Pakistani Prime Minister Benazir Bhutto  U. S. Secretary of Housing and Urban Development Shaun Donovan  Canadian political leader Michael Ignatieff  Pakistani M

In [453]:
# get all verbs in the sentences
verbs = set()
for sentence in sentences:
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            verbs.add(token.lemma_)
            
verbs
# verbs2 = ["earn"]
# # loop on sentences and get the sentences that have verbs woth lemmas in those [earn, have, suspend]
# sentences_with_verbs = []
# for sentence in sentences:
#     doc = nlp(sentence)
#     for token in doc:
#         if token.lemma_ in verbs2:
#             sentences_with_verbs.append(sentence)
#             break
        
# sentences_with_verbs
# [earn, have, suspend]


set()

In [454]:
excludesPerQuestionType = {
    "when": "Times",
    "where": "Locations",
    "who": "Subject",
    "what": "Objects",
    "how": "States"
}

In [471]:
question = "Who is the Costa Rican President that went to Harvard?" 
# answer = "He talked to him to secure the account."
answer = "The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title."
answer = "As this was the 50th Super Bowl, the league emphasized the golden anniversary with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals ( under which the game would have been known as Super Bowl L ), so that the logo could prominently feature the Arabic numerals 50."
answer = "Aly said yesterday that Ziad was good today."
answer = "Politics: U.N. Secretary General Ban Ki-moon; American political leaders John Hancock, John Adams, John Quincy Adams, Rutherford B. Hayes, Theodore Roosevelt, Franklin D. Roosevelt, John F. Kennedy, Al Gore, George W. Bush and Barack Obama; Chilean President Sebastián Piñera; Colombian President Juan Manuel Santos; Costa Rican President José María Figueres; Mexican Presidents Felipe Calderón, Carlos Salinas de Gortari and Miguel de la Madrid; Mongolian President Tsakhiagiin Elbegdorj; Peruvian President Alejandro Toledo; Taiwanese President Ma Ying-jeou; Canadian Governor General David Lloyd Johnston; Indian Member of Parliament Jayant Sinha; Albanian Prime Minister Fan S. Noli; Canadian Prime Ministers Mackenzie King and Pierre Trudeau; Greek Prime Minister Antonis Samaras; Israeli Prime Minister Benjamin Netanyahu; former Pakistani Prime Minister Benazir Bhutto; U. S. Secretary of Housing and Urban Development Shaun Donovan; Canadian political leader Michael Ignatieff; Pakistani Members of Provincial Assembly Murtaza Bhutto and Sanam Bhutto; Bangladesh Minister of Finance Abul Maal Abdul Muhith; President of Puntland Abdiweli Mohamed Ali; U.S. Ambassador to the European Union Anthony Luzzatto Gardner."
ans_nlp = nlp(answer)
splitted_question = question.split(" ")
question_type = splitted_question[0].lower()
question_nlp = nlp(question)
if question_nlp[0].ent_type_ == "DATE":
    question_type = "when"

question_type

'who'

In [472]:
for token in question_nlp:
    print(token.text, '-',token.pos_,'-', token.dep_,'-', token.ent_type_)

    
displacy.render(question_nlp, style="dep")

Who - PRON - attr - 
is - AUX - ROOT - 
the Costa Rican President - PROPN - nsubj - 
that - PRON - nsubj - 
went - VERB - relcl - 
to - ADP - prep - 
Harvard - PROPN - pobj - ORG
? - PUNCT - punct - 


In [473]:
for token in ans_nlp:
    
    print(token.text, '-',token.pos_,'-', token.dep_,'-', token.ent_type_)
        

displacy.render(ans_nlp, style="dep")

Politics - NOUN - ROOT - 
: - PUNCT - punct - 
U.N. Secretary General Ban Ki-moon - PROPN - appos - PERSON
; - PUNCT - punct - 
American political leaders John Hancock - PROPN - conj - PERSON
, - PUNCT - punct - 
John Adams - PROPN - appos - PERSON
, - PUNCT - punct - 
John Quincy Adams - PROPN - conj - PERSON
, - PUNCT - punct - 
Rutherford B. Hayes - PROPN - conj - PERSON
, - PUNCT - punct - 
Theodore Roosevelt - PROPN - conj - PERSON
, - PUNCT - punct - 
Franklin D. Roosevelt - PROPN - conj - PERSON
, - PUNCT - punct - 
John F. Kennedy - PROPN - conj - PERSON
, - PUNCT - punct - 
Al Gore - PROPN - conj - PERSON
, - PUNCT - punct - 
George W. Bush - PROPN - conj - PERSON
and - CCONJ - cc - 
Barack Obama - PROPN - conj - PERSON
; - PUNCT - punct - 
Chilean President Sebastián Piñera - PROPN - conj - PERSON
; - PUNCT - punct - 
Colombian President Juan Manuel Santos - PROPN - conj - PERSON
; - PUNCT - punct - 
Costa Rican President José María Figueres - PROPN - conj - PERSON
; - PUNCT 

### Extracting the entities

In [458]:
def extract_subjects(sentence):
    subjects = {}
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            verbIdx += 1
            subjectFlag = False
            verb = token
            for child in token.children:
                if child.dep_ in ("nsubj", "csubj"):
                    subtree_tokens = [str(t) for t in child.subtree]
                    subjects[token] = (" ".join(subtree_tokens), verbIdx)
                    subjectFlag = True
                elif child.dep_ == "nsubjpass":
                    for child in verb.children:
                        if child.dep_ == "agent" and len(list(child.children)) > 0:
                            subject = [str(t) for t in list(child.children)[0].subtree]
                            subject = " ".join(subject)
                            break
                        else:
                            subject = "Unknown"
                    subjects[verb] = (subject, verbIdx)
                    subjectFlag = True
            if not subjectFlag:  # didn't find a normal subject
                if token.dep_ == "relcl":
                    subject = str(token.head)
                    subjects[token] = (subject, verbIdx)  # should get the subtree of the subject
                elif token.dep_ in ("advcl", "conj"):
                    verb = token.head
                    
                    if verb in subjects:
                        subjects[token] = (subjects[verb][0], verbIdx)
                    else:
                        subjects[token] = ("Unknown", verbIdx)  # replace "Unknown" with a suitable default
                elif token.dep_ == "xcomp":
                    verb = token.head
                    if verb in subjects:
                        subjects[token] = (subjects[verb][0], verbIdx)
                    else:
                        subjects[token] = ("Unknown", verbIdx)
                    for child in verb.subtree:
                        if child.dep_ in ("dobj", "dative", "pobj"):
                            subtree_tokens = [str(t) for t in child.subtree]
                            subjects[token] = (" ".join(subtree_tokens), verbIdx)
                            break
                else:
                    subjects[token] = ("Unknown", verbIdx)
                                        
    # (subject, verbIdx, verb)
    return [(v[0], k, v[1]) for k, v in subjects.items()]             
                            
def extract_objects(sentence):
    objects = []
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            verbIdx += 1
            for child in token.children:
                if child.dep_ in ("dobj", "dative", "attr", "oprd", "acomp","ccomp", "xcomp", "nsubjpass"):
                    subtree_tokens = [str(t) for t in child.subtree]
                    objects.append((" ".join(subtree_tokens), token, verbIdx))
                    
    return objects

def extract_state(sentence):
    states = []
    verbIdx = 0
    for token in sentence:
        if token.pos_ =="VERB" or token.pos_ == "AUX":
            verbIdx += 1
            for child in token.children:
                if child.dep_ == "prep":
                    subtree_tokens = [str(t) for t in child.subtree]
                    states.append(((" ".join(subtree_tokens), token, verbIdx)))
    return states

def extract_time(sentence):
    times = {}
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            verbIdx += 1
            for child in token.subtree:
                if child.ent_type_ == "DATE" or child.ent_type_ == "TIME":
                    times[child.text] = (token, verbIdx)
                    
    return [(k, v[0], v[1]) for k, v in times.items()]

def extract_location(sentence):
    locations = {}
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            verbIdx += 1
            for child in token.subtree:
                if child.ent_type_ in ("GPE", "LOC", "FAC"):
                    locations[child.text] = (token, verbIdx)
                    
    return [(k, v[0], v[1]) for k, v in locations.items()]
                    

def extract_facts(sentence):
    sentence = nlp(sentence)
    states = extract_state(sentence)
    subjects = extract_subjects(sentence)
    objects = extract_objects(sentence)
    times = extract_time(sentence)
    locations = extract_location(sentence)
    print("Subjects: ",subjects, objects)
    print("states: " ,states, "times: ", times,"locations: ", locations)
    
    facts = pd.DataFrame(columns=["Subject", "Relation", "verbIdx", "Objects", "States", "Times", "Locations"])
    
    for subject in subjects: #(Aly, is, 1), (Ziad,is, 2) 
        currentSubject = subject[0]
        verb = subject[1].lemma_
        verbIdx = subject[2]
        mask = (facts['Subject'] != currentSubject) | (facts['Relation'] != verb)
        if mask.all():
            new_row = pd.DataFrame([{"Subject": currentSubject, "Relation": verb, "verbIdx": verbIdx, "Objects": [], "States": [], "Times": [], "Locations": []}])
            facts = pd.concat([facts, new_row], ignore_index=True)

    for obj in objects: #(happy, is, 1), (good, is, 2)
        currentObj = obj[0]
        verb = obj[1].lemma_
        verbIdx = obj[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldObjects = list(facts.loc[mask, "Objects"].values[0])
            oldObjects.append(currentObj)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Objects"] = oldObjects
            
    for state in states:
        currentState = state[0]
        verb = state[1].lemma_
        verbIdx = state[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldStates = list(facts.loc[mask, "States"].values[0])
            oldStates.append(currentState)
            for idx in facts.loc[mask].index:
                facts.at[idx, "States"] = oldStates
            
    for time in times:
        currentTime = time[0]
        verb = time[1].lemma_
        verbIdx = time[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldTimes = list(facts.loc[mask, "Times"].values[0])
            oldTimes.append(currentTime)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Times"] = oldTimes
            
    for location in locations:
        currentLocation = location[0]
        verb = location[1].lemma_
        verbIdx = location[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldLocations = list(facts.loc[mask, "Locations"].values[0])
            oldLocations.append(currentLocation)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Locations"] = oldLocations
            
    facts = facts.drop(columns=["verbIdx"])
    return facts

In [459]:
extract_facts(answer)

Subjects:  [('Aly', said, 1), ('Ziad', was, 2)] [('that Ziad was good today', said, 1), ('good', was, 2)]
states:  [] times:  [('yesterday', said, 1), ('today', was, 2)] locations:  []


,Subject,Relation,Objects,States,Times,Locations
0,Aly,say,[that Ziad was good today],[],[yesterday],[]
1,Ziad,be,[good],[],[today],[]


In [460]:
questionDF = extract_facts(question)

Subjects:  [('the Costa Rican President that went to Harvard', is, 1), ('that', went, 2)] [('Who', is, 1)]
states:  [('to Harvard', went, 2)] times:  [] locations:  []


In [461]:
def join_sentences_facts(sentences):
    all_facts = pd.DataFrame(columns=["Subject", "Relation", "Objects", "States", "Times", "Locations"])
    for sentence in sentences:
        facts = extract_facts(sentence)
        all_facts = pd.concat([all_facts, facts])
    all_facts = all_facts.groupby(["Subject", "Relation"], as_index=False).agg({
        "Objects": lambda x: [item for sublist in x for item in sublist],
        "States": lambda x: [item for sublist in x for item in sublist],
        "Times": lambda x: [item for sublist in x for item in sublist],
        "Locations": lambda x: [item for sublist in x for item in sublist]
    })
    return all_facts

In [462]:
factsDF = join_sentences_facts(sentences)

Subjects:  [] []
states:  [] times:  [] locations:  []


In [463]:
factsDF

,Subject,Relation,Objects,States,Times,Locations


In [464]:
# iterate the dataframe and change the subject to [Subject] and the relation to [Relation]
def change_subject_relation(factsDF, isQuestion = True):
    if not isQuestion:
        factsDF = factsDF[~((factsDF["Subject"] == "Unknown") & (factsDF["Objects"].apply(len) == 0) & (factsDF["States"].apply(len) == 0) & (factsDF["Times"].apply(len) == 0) & (factsDF["Locations"].apply(len) == 0))]

    for index, row in factsDF.iterrows():
        factsDF.loc[index, "Subject"] = [row['Subject']]
        factsDF.loc[index, "Relation"] = [row['Relation']]
    return factsDF

In [465]:
newFactsDF = change_subject_relation(factsDF, False)
#save in csv
newFactsDF.to_csv("facts2.csv")

In [466]:
newFactsDF

,Subject,Relation,Objects,States,Times,Locations


In [467]:
newQuestionDF = change_subject_relation(questionDF, True)
newQuestionDF

,Subject,Relation,Objects,States,Times,Locations
0,[the Costa Rican President that went to Harvard],[be],[Who],[],[],[]
1,[that],[go],[],[to Harvard],[],[]


In [468]:
def similarity(factRow, questionRow, column):
    if len(factRow[column]) == 0 or len(questionRow[column]) == 0 or factRow[column] == ["Unknown"] or questionRow[column] == ["Unknown"]:
        return 0
    columnString = " ".join(factRow[column])
    questionString = " ".join(questionRow[column])
    embeddingFact = model.encode(columnString)
    embeddingQuestion = model.encode(questionString)
    return util.cos_sim(embeddingFact, embeddingQuestion)

In [469]:
def cost_function(factsDf, questionFact, excludeColumns=[]):
    score = 0
    maxFactIdx = 0
    columnNames = ["Subject","Relation", "Objects", "States", "Times", "Locations"]
    for column in excludeColumns:
        columnNames.remove(column)
    for factIdx, factRow in factsDf.iterrows():
        currScore = 0
        for _, questionRow in questionFact.iterrows():
            if len(factRow[excludeColumns[0]]) == 0:
                continue
            for column in columnNames:
                currScore += similarity(factRow, questionRow, column)
        if currScore > score:
            score = currScore
            maxFactIdx = factIdx
    return maxFactIdx, score
        

In [470]:
correctIdx, _ = cost_function(factsDF, questionDF, excludeColumns=[excludesPerQuestionType[question_type]])
WhenAnswer = factsDF.loc[correctIdx, excludesPerQuestionType[question_type]]
if WhenAnswer == []:
    WhenAnswer = factsDF.loc[correctIdx, "States"]    
" ".join(WhenAnswer)

KeyError: 0

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
emb1 = model.encode("the champion of the National Football League ( NFL )")
emb2 = model.encode("place")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim.item())

Cosine-Similarity: 0.014753999188542366


In [ ]:
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')